## Jupyter Setup

In [ ]:
%reset

In [ ]:
from ase import Atoms
from ase.io import read, write

import matplotlib.pyplot as plt
import nglview as nv
import numpy as np
import os
import re

%matplotlib inline

## Preparation and Input

In [ ]:
# defining universal constants
c = 299792458
mu_0 = 1.25663706212 * 1e-6
eps_0 = np.reciprocal(np.square(c) * mu_0)
k_B = 1.380649 * 1e-23
h =  6.62607015 * 1e-34

In [ ]:
# read atomic weights from official NIST data - see http://physics.nist.gov/Comp for source
symbols, isotopic_comps, masses = [], [], []
atomic_weights_file = '../data/NIST_atomic_weights.txt'

with open(atomic_weights_file, 'r') as f:
    atomic_weights_data = [l.strip() for l in f.readlines() if len(l) > 1]
    
for l in atomic_weights_data:
    elements = l.split(' =')
    if elements[0] == 'Atomic Symbol':
        symbols.append(elements[1].strip())
    elif elements[0] == 'Isotopic Composition':
        isotopic_comp = 1E-5 # assume v.low isotopic composition if no value given to avoid NaN
        if elements[1]:
            isotopic_comp = float(re.findall('[0-9]+\.?[0-9]*', elements[1])[0])
        isotopic_comps.append(isotopic_comp)
    elif elements[0] == 'Relative Atomic Mass':
        mass = float(re.findall('[0-9]+.[0-9]+', elements[1])[0])
        masses.append(float(mass))

atomic_mass = {symbol: mass for (symbol, mass, isotopic_comp) in zip(symbols, masses, isotopic_comps) \
               if isotopic_comp > 0.5} # only consider the most prevalent isotope for simplicity

In [ ]:
work_dir = %pwd

In [ ]:
# Choose and name sample molecule

# name, data_dir = 'Ethane', '../data/cp2k_vibr_c2h6/raman/'
# name, data_dir = '7-AGNR (3 units)', '../data/cp2k_vibr_3units_7agnr/raman/'
# name, data_dir = '7-AGNR (6 units)', '../data/cp2k_vibr_6units_7agnr/raman/'
# name, data_dir = '7-AGNR (12 units)', '../data/cp2k_vibr_12units_7agnr/raman/'
# name, data_dir = '7-AGNR (12 units) "Bite" Defect', '../data/cp2k_vibr_12units_7agnr_defect_bite/raman/'
# name, data_dir = '7-AGNR (12 units) Nitrogen Doping Defect', '../data/cp2k_vibr_12units_7agnr_defect_N/raman/'
name, data_dir = '7-AGNR (12 units) Nitrogen+Boron Doping Defect', '../data/cp2k_vibr_12units_7agnr_defect_N_B/raman/'

In [ ]:
cp2k_in_file = data_dir + 'cp2k.inp'
cp2k_out_file = data_dir + 'cp2k.out'

In [ ]:
with open(cp2k_in_file, 'r') as f:
    cp2k_inp_data = [l.strip() for l in f.readlines()]

with open(cp2k_out_file, 'r') as f:
    cp2k_out_data = [l.strip() for l in f.readlines()]

In [ ]:
for l in cp2k_inp_data:
    elements = l.split()
    if elements:
        if elements[0] == 'COORD_FILE_NAME':
            coord_file = data_dir + elements[1].strip('./')
        elif elements[0] == 'DX':
            dx = float(elements[1])

In [ ]:
with open(coord_file, 'r') as f:
    coord_data = [l.strip() for l in f.readlines()]
    
n = int(coord_data[0])
opt_steps = int(len(coord_data) / (n+2))
assert (len(coord_data) % opt_steps == 0)

coord_data = coord_data[(opt_steps-1) * (n+2):]

atoms = np.zeros(n, dtype=np.str)
positions = np.zeros((n, 3))

for i in range(len(coord_data)-2):
    atoms[i], *positions[i] = coord_data[i+2].split()

molecule = Atoms(atoms, positions)    

In [ ]:
# array of 1 / sqrt(atomic_mass) for calculating mass-weighted normal coordinate polarization derivatives below
atom_mass_inv_sqrt = np.sqrt(np.reciprocal([atomic_mass.get(atom) for atom in atoms]))

In [ ]:
# display molecule for visual check
nv.show_ase(molecule)

In [ ]:
# write molecule image to file for quick check if required
# write('visual_'+name+'.png', molecule) # PNG file
# write('visual_'+name+'.eps', molecule) # EPS file

## Parsing CP2K Output

In [ ]:
normal_freqs_list = []
normal_intens_list = []
normal_redmasses_list = []
normal_frcconsts_list = []

task_dict = {}
dim_dict = {'X': 0, 'Y': 1, 'Z': 2}
step_dict = {'+': 0, '-': 1}

atom_displacements, atom_labels = [], []
hessian_elements, hessian_labels = [], []
for a in range(1, 1+n):
    atom_displacements.append([])
    atom_labels.append(str(a))
    hessian_elements.extend([[], [], []])
    hessian_labels.extend([str(a*3-2), str(a*3-1), str(a*3)])

hessian_section, normal_mode_section = False, False

for l in cp2k_out_data:
    elements = l.split()
    if not normal_mode_section and not hessian_section: # vibrational analysis section
        if len(elements) > 1:
            if elements[0] == 'REPLICA|':
                if elements[-2] == 'groups':
                    num_groups = int(elements[-1])
                elif elements[-2] == 'group':
                    group_size = int(elements[-1])
                task_counter = num_groups * [1]
            elif elements[0] == 'VIB|':
                if elements[1] == 'REPLICA':
                    replica_num = int(elements[3].strip('-'))
                    atom_num = int(elements[9]) - 1
                    dim_num = dim_dict[elements[11]]
                    step_num = step_dict[elements[12]]
                    task_num = task_counter[replica_num-1]
                    task_counter[replica_num-1] += 1                    
                    task_key = str(replica_num) + '_' + str(task_num)
                    task_dict[task_key] = [atom_num, dim_num, step_num]                    
                elif elements[1] == 'Hessian':
                    hessian_section = True
                elif elements[1] == 'NORMAL':
                    normal_mode_section = True
    
    elif hessian_section:
        if len(elements) > 1:
            if elements[0] in hessian_labels and elements[1] not in hessian_labels:
                hessian_row = int(elements[0]) - 1
                hessian_elements[hessian_row].extend(elements[2:])
            elif elements[0] == 'VIB|' and elements[1] == 'Cartesian':
                    hessian_section = False
            
    elif normal_mode_section:
        if elements:
            if elements[0] == "VIB|Frequency":
                [normal_freqs_list.append(float(freq)) for freq in elements[2:]]
            elif elements[0] == "VIB|Intensities":
                [normal_intens_list.append(float(freq)) for freq in elements[1:]]
            elif elements[0] == "VIB|Red.Masses":
                [normal_redmasses_list.append(float(freq)) for freq in elements[2:]]
            elif elements[0] == "VIB|Frc":
                [normal_frcconsts_list.append(float(freq)) for freq in elements[3:]]
            elif elements[0] == "COUNTER":
                normal_mode_section = False
            elif elements[0] in atom_labels:
                atom_number = int(elements[0]) - 1
                atom_displacements[atom_number].extend(elements[2:])
                
inv_task_dict = {str(v): k for k, v in task_dict.items()}                

assert(len(atom_displacements[0]) == len(atom_displacements[-1]))


hessian = np.asarray(hessian_elements).astype(np.float)

num_normal_modes = int(len(atom_displacements[0]) / 3)
                
normal_freqs = np.asarray(normal_freqs_list)
normal_intens = np.asarray(normal_intens_list)
normal_redmasses = np.asarray(normal_redmasses_list)
normal_frcconsts = np.asarray(normal_frcconsts_list)

normal_displacements = np.ndarray((n, num_normal_modes, 3))
for a in range(len(atom_labels)):
    normal_displacements[a] = np.asarray(atom_displacements[a]).reshape(-1, 3).astype(np.float)

assert(normal_freqs.shape == normal_intens.shape == normal_redmasses.shape == normal_frcconsts.shape)

In [ ]:
def atoi(snip):
    return int(snip) if snip.isdigit() else snip

def natural_key(name):
    return [atoi(n) for n in re.split(r'(\d+)', name)]

In [ ]:
data_file_list = [f for f in os.listdir(data_dir) if f.endswith('.data')]
data_file_list.sort(key=natural_key)

In [ ]:
###  assigning polarizability tensors to proper atoms, dimensions,
### and distortion steps according to cp2k.out:

dx_steps = len(step_dict) # = 2 for central finite differences 3-point scheme
dims =  len(dim_dict)# = 3 for 3D
polarizabilities = np.ndarray((n, dims, dx_steps, dims, dims))
for file in data_file_list:
    [task_key] = re.findall('[0-9]+_[0-9]+', file)
    if task_key in task_dict:
        atom_num, dim_num, step_num = task_dict.get(task_key)
        file_path = data_dir + file
        with open(file_path, 'r') as f:
            tensor_data = [l.strip() for l in f.readlines()]
        au_section, ang_section = False, False
        for l in tensor_data:
            elements = l.split()
            if elements:
                if elements[2] == '(atomic':
                    au_section, ang_section = True, False
                elif elements[2] == '(Angstrom^3):':
                    au_section, ang_section = False, True
                elif elements[0] == 'xx,yy,zz' and ang_section:
                    polarizabilities[atom_num][dim_num][step_num][0][0] = elements[1]
                    polarizabilities[atom_num][dim_num][step_num][1][1] = elements[2]
                    polarizabilities[atom_num][dim_num][step_num][2][2] = elements[3]
                elif elements[0] == 'xy,xz,yz' and ang_section:
                    polarizabilities[atom_num][dim_num][step_num][0][1] = elements[1]
                    polarizabilities[atom_num][dim_num][step_num][0][2] = elements[2]
                    polarizabilities[atom_num][dim_num][step_num][1][2] = elements[3]
                elif elements[0] == 'yx,zx,zy' and ang_section:
                    polarizabilities[atom_num][dim_num][step_num][1][0] = elements[1]
                    polarizabilities[atom_num][dim_num][step_num][2][0] = elements[2]
                    polarizabilities[atom_num][dim_num][step_num][2][1] = elements[3]

## Calculating Raman Tensor Placzek Invariants

In [ ]:
# FDM 3-point first derivative of polarizability tensors:

# dx in cp2k is in Bohr, polarizability tensors are in Angstrom^3:
bohr2ang = 0.5291772109
factor = 1 / (2 * dx * bohr2ang)

# polarizability tensor derivatives w.r.t. cartesian coordinates in Angstrom^2:
polariz_dxyz = factor * (polarizabilities[:, :, 0, :, :] - polarizabilities[:, :, 1, :, :])

In [ ]:
# Calculating Raman tensor from polarizability tensor derivatives, normal mode displacements, atomic masses

polariz_dq = np.einsum('ad...,akd,a->k...', polariz_dxyz, normal_displacements, atom_mass_inv_sqrt)

In [ ]:
# Placzek tensor invariants

# mean polarizability squared:
a_sq = np.square(np.trace(polariz_dq, 0, 2) / 3).reshape((-1,1))

# anisotropy:
gamma_sq = np.zeros((num_normal_modes,1))

# asymmetric anisotropy:
delta_sq = np.zeros_like(gamma_sq)

for k in range(num_normal_modes):
    gamma_sq[k] = 0.5  * (np.square(polariz_dq[k][0][0] - polariz_dq[k][1][1])
                         +np.square(polariz_dq[k][1][1] - polariz_dq[k][2][2])
                         +np.square(polariz_dq[k][2][2] - polariz_dq[k][0][0])) \
                         +3 * (np.square(polariz_dq[k][0][1])
                              +np.square(polariz_dq[k][1][2])
                              +np.square(polariz_dq[k][2][0]))
    delta_sq[k] = 0.75 * (np.square(polariz_dq[k][0][1] - polariz_dq[k][1][0])
                         +np.square(polariz_dq[k][0][2] - polariz_dq[k][2][0])
                         +np.square(polariz_dq[k][1][2] - polariz_dq[k][2][1]))

In [ ]:
# setup variables for plotting
I_raman = np.zeros((num_normal_modes,3))
dsigma_domega = np.zeros_like(I_raman)
cs_raman = np.zeros_like(I_raman)

## Plotting and Output Function Definitions

In [ ]:
def set_size(width='thesis', fraction=1, subplots=(1, 1)):
    """Set figure dimensions to avoid scaling in LaTeX.

    credit to Jack Walton, see https://jwalton.info//Embed-Publication-Matplotlib-Latex/

    Parameters
    ----------
    width: float or string
            Document width in points, or string of predined document type
    fraction: float, optional
            Fraction of the width which you wish the figure to occupy
    subplots: array-like, optional
            The number of rows and columns of subplots.
    Returns
    -------
    fig_dim: tuple
            Dimensions of figure in inches
    """
    if width == 'thesis':
        width_pt = 426.79135
    elif width == 'beamer':
        width_pt = 307.28987
    else:
        width_pt = width

    # Width of figure (in pts)
    fig_width_pt = width_pt * fraction
    # Convert from pt to inches
    inches_per_pt = 1 / 72.27

    # Golden ratio to set aesthetic figure height
    # https://disq.us/p/2940ij3
    golden_ratio = (5**.5 - 1) / 2

    # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    fig_height_in = fig_width_in * golden_ratio * (subplots[0] / subplots[1])

    return (fig_width_in, fig_height_in)

In [ ]:
def plot_intensity(title=' ',  start_idx=0, end_idx=None, normal_freqs=normal_freqs, I_raman=I_raman, width='thesis', color='darkblue', linewidth=5.0):
    
    plt.style.use('seaborn')
    tex_fonts = {
        # Use LaTeX to write all text
        "text.usetex": True,
        "font.family": "serif",
        # Use 10pt font in plots, to match 10pt font in document
        "axes.labelsize": 12,
        "font.size": 12,
        # Make the legend/label fonts a little smaller
        "legend.fontsize": 10,
        "xtick.labelsize": 10,
        "ytick.labelsize": 10
    }
    plt.rcParams.update(tex_fonts)
    
    plt.subplots(1, 1, figsize=set_size(width))
    plt.xlabel('Raman shift [$cm^{-1}$]')
    plt.ylabel('Raman intensity [$\AA^4  amu^{-1}$]')
    
    plt.vlines(normal_freqs[start_idx:end_idx],
               np.zeros_like(normal_freqs[start_idx:end_idx]),
               I_raman[start_idx:end_idx, 0],
               color=color,
               linewidth=linewidth)
    
    plt.hlines(0, *plt.xlim(left=0))
    plt.ylim(bottom=-20)
    plt.xlim([-50, 1.05 * normal_freqs[end_idx if end_idx else -1]])
    
    plt.ticklabel_format(axis='y', style='sci', scilimits=(0,3), useOffset=False, useMathText=True)
    
#     plt.title(title, pad=20, wrap=True)
    plt.tight_layout()
    
    title = re.sub('\W+','', title)
    plt.savefig(title + '.pdf', bbox_inches='tight')
    plt.savefig(title + '.png', bbox_inches='tight')
    plt.savefig(title + '.eps', bbox_inches='tight')

In [ ]:
def plot_intensities(name=name, start_idx=0, end_idx=None, normal_freqs=normal_freqs, I_raman=I_raman, width='thesis', color='darkblue', linewidth=5.0):
    plt.style.use('seaborn')
    tex_fonts = {
        # Use LaTeX to write all text
        "text.usetex": True,
        "font.family": "serif",
        # Use 10pt font in plots, to match 10pt font in document
        "axes.labelsize": 12,
        "font.size": 12,
        # Make the legend/label fonts a little smaller
        "legend.fontsize": 10,
        "xtick.labelsize": 10,
        "ytick.labelsize": 10
    }
    plt.rcParams.update(tex_fonts)
    
    n_subplots = 3
    title = 'Computed ' + name
    
    fig, axs = plt.subplots(n_subplots, figsize=set_size(width, subplots=(3,1)))
    
    for i in range(n_subplots):    
        axs[i].set_title(title)
        axs[i].vlines(normal_freqs[start_idx:end_idx],
                      np.zeros_like(normal_freqs[start_idx:end_idx]),
                      I_raman[start_idx:end_idx, i],
                      color=color,
                      linewidth=linewidth)
        
    for ax in axs.flat:
        ax.hlines(0, *plt.xlim(left=0))
        ax.set(xlabel='Raman shift [$cm^{-1}$]', ylabel='Raman intensity [$\AA^4  amu^{-1}$]')
        ax.label_outer()
        ax.set_ylim(-20, axs[0].get_ylim()[1])
        ax.set_xlim([-50, 1.05 * normal_freqs[end_idx if end_idx else -1]])
        ax.ticklabel_format(axis='y', style='sci', scilimits=(0,3), useOffset=False, useMathText=True)


        
    axs[0].set_title(title + " total Raman intensity $I(\perp^s + \parallel^s)$", pad=20, wrap=True)
    axs[1].set_title(title + " perpendicular Raman intensity $I(\perp^s)$", pad=20, wrap=True)
    axs[2].set_title(title + " parallel Raman intensity $I(\parallel^s)$", pad=20, wrap=True)
    plt.tight_layout()

    title = re.sub('\W+','', title)
    plt.savefig(title + 'Ramanintensity.pdf', bbox_inches='tight')
    plt.savefig(title + 'Ramanintensity.png', bbox_inches='tight')
    plt.savefig(title + 'Ramanintensity.eps', bbox_inches='tight')

In [ ]:
def plot_cross_section(title=' ', start_idx=0, end_idx=None, normal_freqs=normal_freqs, cs_raman=cs_raman, width='thesis', color='darkblue', linewidth=5.0):
    plt.style.use('seaborn')
    tex_fonts = {
        # Use LaTeX to write all text
        "text.usetex": True,
        "font.family": "serif",
        # Use 10pt font in plots, to match 10pt font in document
        "axes.labelsize": 12,
        "font.size": 12,
        # Make the legend/label fonts a little smaller
        "legend.fontsize": 10,
        "xtick.labelsize": 10,
        "ytick.labelsize": 10
    }
    plt.rcParams.update(tex_fonts)
    
    plt.subplots(1, 1, figsize=set_size(width))
    plt.xlabel('Raman shift [$cm^{-1}$]')
    plt.ylabel('Raman cross-section [arb. units]')

    plt.vlines(normal_freqs, np.zeros_like(normal_freqs), cs_raman[:, 0], color=color, linewidth=linewidth)
    plt.hlines(0, *plt.xlim(left=0))
    
    plt.ylim([-5,105])
    plt.xlim([-50, 1.05 * normal_freqs[end_idx if end_idx else -1]])
    
#     plt.title(title, pad=20, wrap=True)
    plt.tight_layout()
    
    title = re.sub('\W+','', title)
    plt.savefig(title + '.pdf', bbox_inches='tight')
    plt.savefig(title + '.png', bbox_inches='tight')
    plt.savefig(title + '.eps', bbox_inches='tight')

In [ ]:
def plot_cross_sections(name=name, start_idx=0, end_idx=None, normal_freqs=normal_freqs, cs_raman=cs_raman, width='thesis', color='darkblue', linewidth=5.0):
    
    assert start_idx >=0
    if end_idx:
        assert start_idx < end_idx and end_idx < len(normal_freqs)
    
    plt.style.use('seaborn')
    tex_fonts = {
        # Use LaTeX to write all text
        "text.usetex": True,
        "font.family": "serif",
        # Use 10pt font in plots, to match 10pt font in document
        "axes.labelsize": 12,
        "font.size": 12,
        # Make the legend/label fonts a little smaller
        "legend.fontsize": 10,
        "xtick.labelsize": 10,
        "ytick.labelsize": 10
    }
    plt.rcParams.update(tex_fonts)
    
    n_subplots = 3
    title = 'Computed ' + name + ' ($\lambda_L=$ {}nm)'.format(laser_wl)
    
    fig, axs = plt.subplots(n_subplots, figsize=set_size(width, subplots=(3,1)))
    
    for i in range(n_subplots):    
        axs[i].set_title(title)
        axs[i].vlines(normal_freqs[start_idx:end_idx],
                      np.zeros_like(normal_freqs[start_idx:end_idx]),
                      cs_raman[start_idx:end_idx, i],
                      color=color,
                      linewidth=linewidth)
        
    for ax in axs.flat:
        ax.hlines(0, *plt.xlim(left=0))
        ax.set(xlabel='Raman shift [$cm^{-1}$]', ylabel='Raman cross-section [arb. units]')
        ax.label_outer()
        ax.set_ylim([-5,105])
        ax.set_xlim([-50, 1.05 * normal_freqs[end_idx if end_idx else -1]])
        
    axs[0].set_title(title + " total Raman cross-section $\sigma' (\perp^s + \parallel^s)$", pad=20, wrap=True)
    axs[1].set_title(title + " perpendicular Raman cross-section $\sigma' (\perp^s)$", pad=20, wrap=True)
    axs[2].set_title(title + " parallel Raman cross-section $\sigma' (\parallel^s)$", pad=20, wrap=True)
    plt.tight_layout()

    title = re.sub('\W+','', title)
    plt.savefig(title + 'Ramancrosssection.pdf', bbox_inches='tight')
    plt.savefig(title + 'Ramancrosssection.png', bbox_inches='tight')
    plt.savefig(title + 'Ramancrosssection.eps', bbox_inches='tight')
    

## Calculating and Plotting Raman Intensities, Raman Cross-Sections

In [ ]:
# defining experimental parameters

laser_wls = [0, 532, 785] # [nm]
T = 300 # [K] - CP2K default


# normal frequency range
start_idx = 0
# to find normal frequency index where to cut off to match experimental reference plots
# freq_cutoff = 1800 # suitable cutoff for small AGNR experiments (empirical)
freq_cutoff = np.inf # generic cutoff
end_idx = np.where(normal_freqs < freq_cutoff)[0][-1]


### Raman intensities

In [ ]:
# calculate absolute Raman intensity: total, perpendicular & parallel polarized scattering

for k in range(num_normal_modes):
    if normal_freqs[k] > 0:  # there are no physical negative normal frequencies
        I_raman[k] = 45 * a_sq[k] + 7 * gamma_sq[k] + 5 * delta_sq[k],\
                     45 * a_sq[k] + 4 * gamma_sq[k]                  ,\
                                    3 * gamma_sq[k] + 5 * delta_sq[k]
# plot intensities        
plot_intensities(end_idx=end_idx)
plot_intensity(title='Computed '+name+" total Raman intensity $I(\perp^s + \parallel^s)$", end_idx=end_idx)


### Raman Cross-Sections

In [ ]:
# function to calculate differential cross section
def diff_cross_section(I_k, nu_k, laser_wl=0, T=300):
    if laser_wl > 0:
        nu_in = np.reciprocal(laser_wl * 10e-7) # nm2cm = 1e-7
    else:
        nu_in = 0 # limit case
    return np.pi**2 / eps_0**2 * (nu_in - nu_k)**4 * h / (8 * np.pi**2 * c * nu_k) \
           * I_k / 45 * np.reciprocal(1 - np.exp(-h * c * nu_k / (k_B * T)))

In [ ]:
for laser_wl in laser_wls:
    # calculate differential Raman cross-section: total, perpendicular & parallel polarized scattering
    for k in range(num_normal_modes):
        if normal_freqs[k] > 0 and start_idx <= k < (end_idx if end_idx else num_normal_modes): # there are no physical negative normal frequencies
            dsigma_domega[k] = diff_cross_section(I_raman[k, 0], normal_freqs[k], laser_wl, T), \
                               diff_cross_section(I_raman[k, 1], normal_freqs[k], laser_wl, T), \
                               diff_cross_section(I_raman[k, 2], normal_freqs[k], laser_wl, T)
    cs_raman = dsigma_domega / np.max(dsigma_domega) * 100 # scale arbitrary units to 0-100

    # plot cross-sections
    plot_cross_sections(end_idx=end_idx, cs_raman=cs_raman)
    plot_cross_section(title='Computed '+name+" ($\lambda_L=${}nm) total Raman cross-section $\sigma'(\perp^s + \parallel^s)$".format(laser_wl), end_idx=end_idx, cs_raman=cs_raman)

## Verification Example for Ethane ($\text{C}_2\text{H}_6$)

In [ ]:
# for verification C2H6
# Porezag and Pederson (1996), Physical Review B, 54(11):7830–7836, doi:10.1103/PhysRevB.54.7830
ref_normal_freqs = np.asarray([301, 1016, 1246, 1449, 1552, 3043, 3175])
ref_I_raman = np.asarray([0.0, 13.4, 0.6, 0.2, 17.8, 302.0, 290.0]).reshape((-1,1))

title = 'Reference Ethane experimental total Raman intensity $I(\perp^s + \parallel^s)$'
plot_intensity(title, start_idx=0, end_idx=None, normal_freqs=ref_normal_freqs, I_raman=ref_I_raman)